# Reading Public Kepler Data
When accessing many gigabytes of data on S3, significant speed up can be gained by retrieving the data from the [S3 buckets](https://aws.amazon.com/s3/) (Amazon Web Service's secure filestore system) instead of pulling it from MAST in Baltimore, Maryland. 

This tutorial demonstrates how to do the following:

- Use boto3, Amazon package for interacting with AWS services
- Find the name of the Kepler data of interest
- Transfer data from the bucket to the local file system

In [4]:
import boto3

The Kepler science data are available in the public [s3://stpubdata/kepler/public](https://registry.opendata.aws/kepler/) S3 bucket on AWS.  We will use files in this bucket to demo transfering and reading a FITS formatted data file from an AWS S3 Bucket.

To start, we will use the boto3 software to download the manifest file for the Kepler S3 public bucket, created by Kepler's archive, MAST. Listing all the files in a public bucket is not simple, so this file provides a way around that limitation. We will download the manifest.txt.gz file to our local file system using [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-download-file.html).  We do this by creating an s3 boto resource and specifying the bucket name, "stpubdata".  Then the download_file method acts like a copy command to transfer the file from the bucket into a filename of your choice in your local file system.


In [5]:
#Choose a resource
s3 = boto3.resource('s3')
#specify a bucket
bucket = s3.Bucket('stpubdata')
#Download the file from the bucket and call it manifest.txt.gz on the local filesystem.
bucket.download_file("kepler/public/manifest.txt.gz", "manifest.txt.gz", ExtraArgs = {"RequestPayer" : "requester"})

ClientError: An error occurred (403) when calling the HeadObject operation: Forbidden

If you get a `ClientError` with a 403 Forbidden error, likely you do not have your [credentials](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-profiles.html) set up correctly.

If you unzip the file you will see a listing of all the filenames stored in this public bucket. We will now transfer one of those files to our local file system in the same way.

In [6]:
from astroquery.mast import Observations

#Have astroquery attempt to retrieve from AWS first.
Observations.enable_cloud_dataset(provider='AWS')

target = "Kepler-10"

#Do a cone search and find the Kepler long cadence data for your target
obs = Observations.query_object(target,radius="0s")
want = (obs['obs_collection'] == "Kepler") & (obs['t_exptime'] ==1800.0)

#Pick which data you want to retrieve
data_prod = Observations.get_product_list(obs[want])
filt_prod = Observations.filter_products(data_prod, description="Lightcurve Long Cadence (CLC) - Q4")

#Move data from the S3 bucket to the default astroquery location. 
#cloud_only=True means that data will only be retrieved if available on AWS S3
manifest = Observations.download_products(filt_prod, cloud_only=True)

INFO: Using the S3 STScI public dataset [astroquery.mast.core]
INFO: See Request Pricing in https://aws.amazon.com/s3/pricing/ for details [astroquery.mast.core]
INFO: If you have not configured boto3, follow the instructions here: https://boto3.readthedocs.io/en/latest/guide/configuration.html [astroquery.mast.core]


ERROR: Error pulling from S3 bucket: An error occurred (403) when calling the HeadObject operation: Forbidden [astroquery.mast.core]
